In [14]:
#@ IMPORTS

import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from tqdm.auto import tqdm

In [3]:
df = pd.read_csv('../03_Logistic_Regression/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.churn = (df.churn == 'yes').astype(int)

## 

### Split the dataset

In [8]:
df_full_train, df_test = train_test_split(df, test_size=0.2,  random_state=1)

df_train_full = df_train_full.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [17]:
# Columns to keep for training
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

columns = categorical + numerical

### Functions for training and predicting

In [19]:
def train(df_train, y_train, columns, C):
    
    dicts = df_train[columns].to_dict(orient = 'records')
    
    dv = DictVectorizer(sparse = False)
    X_train = dv.fit_transform(dicts)
    
    model = LogisticRegression(solver='liblinear', C=C) # we use max_iter to avoid warnings
    model.fit(X_train, y_train)
    
    return dv, model

def predict(df, dv, model, columns):
    dicts = df[columns].to_dict(orient = 'records')
    
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1] # probabilities only for positive examples
    
    return y_pred

### MODEL FINAL TESTING

with 5 fold evaluation to make sure our model is okay

In [20]:
n_splits = 5

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

for C in tqdm([0.01, 0.1, 1, 10]):
    
    scores = []

    
    for train_idx, val_idx in kfold.split(df_full_train): 

        df_train = df_full_train.iloc[train_idx]
        df_val = df_full_train.iloc[val_idx]

        y_train = df_train.churn.values
        y_val = df_val.churn.values

        dv, model = train(df_train, y_train, columns, C=C)

        y_pred = predict(df_val, dv, model, columns)

        auc = roc_auc_score(y_val, y_pred)

        scores.append(auc)

    print('C=%s, %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

  0%|          | 0/4 [00:00<?, ?it/s]

C=0.01, 0.839 +- 0.009
C=0.1, 0.841 +- 0.007
C=1, 0.841 +- 0.007
C=10, 0.841 +- 0.007


### FINAL MODEL

In [29]:
C = 1.0

dv, model = train(df_full_train, df_full_train.churn.values, columns, C)

y_test = df_test.churn.values

y_pred = predict(df_test, dv, model, columns)

auc = roc_auc_score(y_test, y_pred)

auc

0.8579400803839363

SAVE THE MODEL

In [23]:
import pickle

In [30]:
output_file = f'model_C={C}.bin'

output_file

'model_C=1.0.bin'

In [31]:
f_out = open(output_file, 'wb') # open the file and write binary

pickle.dump((dv, model), f_out) # write the vectorizer and the model, we need them both

f_out.close()

But it's common to forget to close the file so the ```with``` method is preffered. This is shorter and the file will always be closed, when out of the with statement

In [32]:
with open(output_file, 'wb') as f_out:

    pickle.dump((dv, model), f_out)

    #do stuff

#do other stuff

### Load the model

now we read the file, so we use ```'rb'``` instead of ```'wb'``` its important to avoid to overwrite the file creating one with zero bytes

In [34]:
import pickle

In [35]:
model_file = 'model_C=1.0.bin'

In [36]:
with open(model_file, 'rb') as f_in: # now we read the file, its important to avoid to overwrite the file creating one with zero bytes

    (dv, model) = pickle.load(f_in)

In [37]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(solver='liblinear'))

To read this correctly we need scikit learn installed, if we try to load pickle file with the model, but we dont need to import

An example of a customer info:

In [52]:


customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}





In [53]:
X = dv.transform([customer]) # remember that DictVectorizer expects a list

X

array([[ 1.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,
         0.  ,  1.  ,  0.  ,  0.  , 29.85,  0.  ,  1.  ,  0.  ,  0.  ,
         0.  ,  1.  ,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,
         0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ,
         0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  , 29.85]])

In [54]:
model.predict_proba(X)[0,1] # Probabillity of a customer to churn

0.6433013293502078

Making requests ()